In [13]:
import pandas as pd
import quandl as qd
import numpy as np
import html5lib 
import bs4
import pickle

In [2]:
api_key = open("api_key_quandl.txt", "r").read()
df1 = qd.get("FMAC/HPI_ALBOR", authtoken = api_key)
df1.head(10)

,Value
Date,
1975-01-31,19.874592
1975-02-28,20.436467
1975-03-31,20.978854
1975-04-30,21.486776
1975-05-31,21.992207
1975-06-30,22.506271
1975-07-31,22.979011
1975-08-31,23.361365
1975-09-30,23.645515


In [3]:
import html5lib 
fiddy_states = pd.read_html("https://simple.wikipedia.org/wiki/List_of_U.S._states")
fiddy_states

[         0             1               2               3                  4
 0   Sl no.  Abbreviation      State Name         Capital     Became a State
 1        1            AL         Alabama      Montgomery  December 14, 1819
 2        2            AK          Alaska          Juneau    January 3, 1959
 3        3            AZ         Arizona         Phoenix  February 14, 1912
 4        4            AR        Arkansas     Little Rock      June 15, 1836
 5        5            CA      California      Sacramento  September 9, 1850
 6        6            CO        Colorado          Denver     August 1, 1876
 7        7            CT     Connecticut        Hartford    January 9, 1788
 8        8            DE        Delaware           Dover   December 7, 1787
 9        9            FL         Florida     Tallahassee      March 3, 1845
 10      10            GA         Georgia         Atlanta    January 2, 1788
 11      11            HI          Hawaii        Honolulu    August 21, 1959

In [4]:
fiddy_states[0]

,0,1,2,3,4
0,Sl no.,Abbreviation,State Name,Capital,Became a State
1,1,AL,Alabama,Montgomery,"December 14, 1819"
2,2,AK,Alaska,Juneau,"January 3, 1959"
3,3,AZ,Arizona,Phoenix,"February 14, 1912"
4,4,AR,Arkansas,Little Rock,"June 15, 1836"
5,5,CA,California,Sacramento,"September 9, 1850"
6,6,CO,Colorado,Denver,"August 1, 1876"
7,7,CT,Connecticut,Hartford,"January 9, 1788"
8,8,DE,Delaware,Dover,"December 7, 1787"
9,9,FL,Florida,Tallahassee,"March 3, 1845"


In [15]:
np_fiddy_states = np.array(fiddy_states[0]) 
np_fiddy_states[1:,1]
def state_list():
    fiddy_states = pd.read_html("https://simple.wikipedia.org/wiki/List_of_U.S._states")
    np_fiddy_states = np.array(fiddy_states[0]) 
    return np_fiddy_states[1:,1]    

In [23]:
def grab_initial_state_data():
    main_df = pd.DataFrame()
    for abv in state_list():
        query = "FMAC/HPI_" + abv
        df = qd.get(query, authtoken = api_key)
        df.rename(columns = {"Value":abv}, inplace = True)
        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df)
#             print(main_df)
    pickle_out = open("fiddy_states.pickle", "wb")
    pickle.dump(main_df, pickle_out)
    pickle_out.close()
grab_initial_state_data()

In [25]:
pickle_in = open('fiddy_states.pickle', 'rb')
HPI_data = pickle.load(pickle_in)
print(HPI_data)

                    AL          AK          AZ          AR          CA  \
Date                                                                     
1975-01-31   35.695975   34.371830   28.937735   36.738861   15.722312   
1975-02-28   35.896770   34.880945   29.463708   37.098962   15.775202   
1975-03-31   36.133518   35.402217   29.959514   37.416398   15.964942   
1975-04-30   36.425723   35.944703   30.345148   37.677813   16.275547   
1975-05-31   36.695708   36.535272   30.546449   37.921099   16.533017   
1975-06-30   36.823713   37.165797   30.496614   38.159640   16.646799   
1975-07-31   36.747060   37.795450   30.173822   38.351084   16.768340   
1975-08-31   36.511773   38.369702   29.662525   38.450073   16.989530   
1975-09-30   36.255136   38.870586   29.128419   38.428010   17.230749   
1975-10-31   36.139294   39.326955   28.828753   38.355341   17.447896   
1975-11-30   36.247464   39.737259   28.941164   38.377787   17.596355   
1975-12-31   36.592162   40.082874   2

### Here we now depict the pandas version of pickle :)

In [29]:
HPI_data.to_pickle('pickle.pickle')
HPI_data2 = pd.read_pickle('pickle.pickle')
print(HPI_data2)

                    AL          AK          AZ          AR          CA  \
Date                                                                     
1975-01-31   35.695975   34.371830   28.937735   36.738861   15.722312   
1975-02-28   35.896770   34.880945   29.463708   37.098962   15.775202   
1975-03-31   36.133518   35.402217   29.959514   37.416398   15.964942   
1975-04-30   36.425723   35.944703   30.345148   37.677813   16.275547   
1975-05-31   36.695708   36.535272   30.546449   37.921099   16.533017   
1975-06-30   36.823713   37.165797   30.496614   38.159640   16.646799   
1975-07-31   36.747060   37.795450   30.173822   38.351084   16.768340   
1975-08-31   36.511773   38.369702   29.662525   38.450073   16.989530   
1975-09-30   36.255136   38.870586   29.128419   38.428010   17.230749   
1975-10-31   36.139294   39.326955   28.828753   38.355341   17.447896   
1975-11-30   36.247464   39.737259   28.941164   38.377787   17.596355   
1975-12-31   36.592162   40.082874   2